<a href="https://colab.research.google.com/github/vegeta03/colab-notebooks/blob/main/GPT-J-6B/Inference_with_GPT_J_6B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inference with GPT-J-6B

In this notebook, we are going to perform inference (i.e. generate new text) with EleutherAI's [GPT-J-6B model](https://github.com/kingoflolz/mesh-transformer-jax/), which is a 6 billion parameter GPT model trained on [The Pile](https://arxiv.org/abs/2101.00027), a huge publicly available text dataset, also collected by EleutherAI. The model itself was trained on TPUv3s using JAX and Haiku (the latter being a neural net library on top of JAX).

[EleutherAI](https://www.eleuther.ai/) itself is a group of AI researchers doing awesome AI research (and making everything publicly available and free to use). They've also created [GPT-Neo](https://github.com/EleutherAI/gpt-neo), which are smaller GPT variants (with 125 million, 1.3 billion and 2.7 billion parameters respectively). Check out their models on the hub [here](https://huggingface.co/EleutherAI).

NOTE: this notebook requires at least 12.1GB of CPU memory. I'm personally using Colab Pro to run it (and set runtime to GPU - high RAM usage). Unfortunately, the free version of Colab only provides 10 GB of RAM, which isn't enough.

## Install dependencies

We will install Transformers from source for now.

In [12]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Load model and tokenizer

First, we load the model from the hub. We select the "float16" revision, which means that all parameters are stored using 16 bits, rather than the default float32 ones (which require twice as much RAM memory). We also set `low_cpu_mem_usage` to `True` (which was introduced in [this PR](https://github.com/huggingface/transformers/pull/13466)), in order to only load the model once into CPU memory.

Next, we move the model to the GPU and load the corresponding tokenizer, which we'll use to prepare text for the model.

In [13]:
import torch
from transformers import GPTJForCausalLM, AutoTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", low_cpu_mem_usage=False)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

## Inference

Here, we can provide a custom prompt, prepare that prompt using the tokenizer for the model (the only input required for the model are the `input_ids`). We then move the `input_ids` also to the GPU, and use the `.generate()` method to generate tokens autoregressively. Note that this method supports various decoding methods, including beam search and top k sampling. All details can be found in this [blog post](https://huggingface.co/blog/how-to-generate).

In [14]:
prompt = "The Belgian national football team "
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The Belgian national football team  also competed in a tournament of the same name, the UEFA Euro 1984. On 14 September 1983, Belgium played its first European Championship qualifying match against Portugal. The game ended in a 1–1 draw, with D'Olimpia scoring in the second half of extra time. The first match in Euro 1984 took place on 18 June 1984 at the Heysel Stadium in Brussels, Belgium, where the Belgian national team played its first game with ten players (Bogaerts was left out due to injury). It ended in a 1–0 win for the Soviet Union. The Belgian national team then won its first two matches in Euro 1984 (against Israel, with a 3–2 win, and against Iceland, with a 3–0 win). Belgium lost to Italy 1–0 in its next match, before winning the tournament with a 3–0 win against Spain in the final. Belgium finished second in the tournament, losing on goal difference to Italy. The


Note that one of the most interesting properties of large GPT models is that they are capable of so-called "few-shot learning". This means that, given only a few examples in a text prompt, the model is capable of quickly generalizing to new, unseen examples. So you can use this model for example to do few-shot text classification, as follows:

In [15]:
prompt = """
Sentence: This movie is very nice.
Sentiment: positive

#####

Sentence: I hated this movie, it sucks.
Sentiment: negative

#####

Sentence: This movie was actually pretty funny.
Sentiment: positive

#####

Sentence: This movie could have been better.
Sentiment: """
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sentence: This movie is very nice.
Sentiment: positive

#####

Sentence: I hated this movie, it sucks.
Sentiment: negative

#####

Sentence: This movie was actually pretty funny.
Sentiment: positive

#####

Sentence: This movie could have been better.
Sentiment:  positive negative

#####

Sentence: I enjoyed this movie overall.
Sentiment:  negative positive

#####

Sentence: I saw this movie because I heard it was good.
Sentiment: negative positive

#####

Sentence: I really hated this movie.
Sentiment:  negative negative

#####

Sentence: I felt this movie was boring.
Sentiment:  positive negative

#####

Sentence: I really enjoyed this movie.
Sentiment:  negative positive

#####


Another note: as GPT-J-6B was trained on The Pile (which includes a lot of Github code), the model is capable of performing code generation (similar to OpenAI's Codex model). Here's an example:

In [16]:
prompt = """Instruction: Generate a Python function that lets you reverse a list.

Answer: """
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruction: Generate a Python function that lets you reverse a list.

Answer: 
def reverse(lst):

    return [rev[0] for rev in list(reversed(lst) )]

How to Test it:
>>> def test_reverse():
def reverse(lst):
        return [rev[0] for rev in list(reversed(lst) )]
    def test_reverse():
        return None
    print test_reverse() 

A:

You could use reversed and list comprehension, like this:
>>> lst = ['a','b','c','d','e','f','g']
>>> reversed(list(lst))
['g','f','e','d','c','b','a']
# if they want to use it later...



## Batched inference

You can also perform inference on batches of prompts, as follows:

In [22]:
# this is a single input batch with size 3
texts = ["Once there was a man ", "The weather today will be ", "The best football player in the world is "]

tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
with torch.no_grad():
    generated_ids = model.generate(**encoding, do_sample=True, temperature=0.9, max_length=50)
generated_texts = tokenizer.batch_decode(
    generated_ids, skip_special_tokens=True)

for text in generated_texts:
  print("---------")
  print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------
Once there was a man  
Who had a golden tooth  
And his name was Lek.  
One day, he went to the market  
And bought some honey.  
With a
---------
The weather today will be 
perfect for some more fun!  

For those of you who did not receive your invitation and/or who chose not to attend 
the happy hour and conference reception please go to the
---------
The best football player in the world is ____________, and here’s why.

For years there was debate about whether Lionel Messi or Cristiano Ronaldo was the best player in the world. The question was complicated by the fact that
